In [ ]:
import numpy as np
import csv
from google.colab import drive
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
csv_file_path = "/content/gdrive/My Drive/train_telugu_news.csv"


In [ ]:


# Initialize an empty list to store concatenated values
concatenated_values = ""

# Open the CSV file in read mode
with open(csv_file_path, 'r') as csvfile:
  # Create a CSV reader object
  csv_reader = csv.reader(csvfile)

  # Skip the header row (optional, adjust if your CSV has a header)
  next(csv_reader, None)

  # Iterate through each row in the CSV
  for row in csv_reader:
    # Extract the value from column 3 (index 2)
    value = row[3]
    # Append the value to the list
    concatenated_values = concatenated_values + value


# Print the concatenated values (optional)
#print(concatenated_values[:1000])

# Alternatively, you can join the values with a separator
# joined_values = ", ".join(concatenated_values)
# print(joined_values)

In [ ]:
concatenated_values[0:100]

'భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ వ్యవహారాలపై భారత రిజర్వు బ్యాంకు (ఆర్\u200cబిఐ) ఒక కన్నేసిం'

In [ ]:

cleaned_text = concatenated_values.replace("\u200c", "")


In [ ]:
cleaned_text[0:100]

'భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ వ్యవహారాలపై భారత రిజర్వు బ్యాంకు (ఆర్బిఐ) ఒక కన్నేసింద'

In [ ]:
len(cleaned_text)

24521128

In [ ]:
cleaned_text = cleaned_text[0:200000]

In [ ]:
cleaned_text

'భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ వ్యవహారాలపై భారత రిజర్వు బ్యాంకు (ఆర్బిఐ) ఒక కన్నేసింది. ఐడిబిఐ బ్యాంకును ఆర్బిఐ వాచ్లి్స్టలో చేర్చినట్టుగా తెలిసింది. నికర మొండిపద్దుల మొత్తం 6 శాతం దాటడం, వరసగా రెండేళ్ల పాటు నష్టాలను  ప్రకటించడం, కాపిటల్ అడెక్వసీ నిర్దేశిత ప్రమాణాల కంటే తగ్గడం... ఈ సందర్భాల్లో బ్యాంకులను ఆర్బిఐ వాచ్ లిస్ట్లో చేరుస్తుంది. తమ బ్యాంకుకు సంబంధించి ఆర్బిఐ ప్రాంప్ట్ కరెక్టీవ్ యాక్షన్ (పిసిఎ) చేపట్టినట్టు ఐడిబిఐ వెల్లడించింది. గతంలో ఓవర్సీస్ బ్యాంక్, యునైటెడ్ బ్యాంక్కు సంబంధించి  కూడా ఆర్బిఐ పిసిఎ చేపట్టింది. పిసిఎ వల్ల బ్యాంకుపై ఆర్థికంగా భారం పడదనీ, దీనివల్ల అంతర్గత నియంత్రణలు మెరుగుపడటంతో పాటు, కార్యకలాపాలు మెరుగవుతాయని ఐడిబిఐ వివరించింది. ఆర్బిఐ ఆదేశాలకు అనుగుణంగా ఇప్పుడు మూలధనాన్ని పొదుపుగా వినియోగించడంపై  ఐడిబిఐ దృష్టి సారించాల్సి ఉంటుంది.ఇందులో భాగంగా పరపతిపై నియంత్రణలు విధించడం, నియామకాలను నిలిపేయడం, కొత్త పెట్టుబడి పథకాలను పక్కన బెట్టడంవంటి చర్యలు తీసుకోవాల్సి ఉంటుం ది. బ్యాంకు సరైన రీతిలో స్పందించని పక్షంలో మరో బలమైన బ్యాంకులో  విలీనానికి  సిద్ధం కావ

In [ ]:
def remove_non_telugu_chars(text):
  """Removes all characters except alphanumeric characters (a-z), space, ?, and \n from a text string.

  Args:
      text: The input text string.

  Returns:
      The text string with all special characters removed.
  """

  disallowed_chars = set("abcdefghijklmnopqrstuvwxyz0123456789'`()-/:@!.\xa0")
  filtered_text = ''.join([char for char in text if char not in disallowed_chars])
  return filtered_text

In [ ]:
cleaned_text = remove_non_telugu_chars(cleaned_text)

In [ ]:
vocab_text = sorted(list(set(cleaned_text)))

ctoi = {k:i for i,k in enumerate(vocab_text)}
ctoi
itoc = {i:k for i,k in enumerate(vocab_text)}
vocab_size = len(vocab_text)


In [ ]:
itoc

In [ ]:
vocab_size

69

In [ ]:
# tokenize routines
encode = lambda x: [ctoi[c] for c in x ]
decode = lambda x: ''.join([itoc[c] for c in x])

In [ ]:
# tokenize the text
etext = encode(cleaned_text)

In [ ]:
# vectorize text
vtext = tf.constant(etext)

In [ ]:
len(vtext)

194604

In [ ]:
# we already have vocab_size. Lets get some more hyperparams
block_size = 8
batch_size = 32
embed_size = 32
num_heads = 4

In [ ]:
# create Train and Val data
n = int(0.9 * len(vtext))
train_data = vtext[:n]
val_data = vtext[n:]

In [ ]:
tf.random.set_seed(1234)
def get_batch(split):
  data = train_data if split == 'train' else val_data
  random_int = tf.random.uniform(shape=(batch_size, ), maxval=len(data) - block_size, minval=0, dtype=tf.int32)
  x = tf.stack([data[i:i+block_size] for i in random_int])
  y = tf.stack([data[i+1:i+block_size+1] for i in random_int])
  return x, y

In [ ]:
# Single Head implementing Attention.
class Head(tf.keras.layers.Layer):
  def __init__(self, head_size):
    super(Head, self).__init__()
    self.key = tf.keras.layers.Dense(head_size) #n_embed
    self.query = tf.keras.layers.Dense(head_size)
    self.value = tf.keras.layers.Dense(head_size)
    self.tril = tf.ones((block_size, block_size))
    self.tril = tf.linalg.band_part(self.tril, -1, 0)
    self.dropout_layer = tf.keras.layers.Dropout(0.2)



  def call(self, x):
    # Perform custom operations on the input
    B,T,C = x.shape
    k=self.key(x)
    q=self.query(x)
    k_t = tf.transpose(k, perm=(0,2,1))

    wei = q @ k_t * C**-0.5

    mask = tf.equal(self.tril, 0)
    replace_value = -1e1000
    wei = tf.where(mask, replace_value, wei)
    wei = tf.nn.softmax(wei, axis=-1)
    wei = self.dropout_layer(wei, training=True)

    v = self.value(x)
    out = wei @ v

    return out

In [ ]:
class MultiHead(tf.keras.layers.Layer):
  def __init__(self, num_heads, head_size):
    super(MultiHead, self).__init__()
    self.num_heads = num_heads
    self.head_size = head_size
    #self.heads = tf.keras.layers.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.heads = []  # Use a list instead of ModuleList
    for _ in range(num_heads):
      self.heads.append(Head(head_size))

    self.proj = tf.keras.layers.Dense(head_size*num_heads)


  def call(self, x):
    head_outputs = []
    for head in self.heads:
      out = head(x)  # Apply each head to the input
      head_outputs.append(out)
    outputs = tf.concat(head_outputs, axis=-1)
    outputs = self.proj(outputs)
    return outputs

In [ ]:
# Feed Forward Layer which goes into attn block.
class FFLayer(tf.keras.layers.Layer):
  def __init__(self, n_embed):
    super().__init__()
    self.n_embed = n_embed
    self.net = tf.keras.layers.Dense(4*n_embed,activation="relu") #n_embed
    self.proj = tf.keras.layers.Dense(n_embed)
    self.dropout_layer = tf.keras.layers.Dropout(0.2)


  def call(self, x):
    out = self.net(x)
    out = self.proj(out)
    out = self.dropout_layer(out, training=True)
    return out

In [ ]:
class AttnBlock(tf.keras.layers.Layer):
  def __init__(self, n_embed, n_head):
    super().__init__()
    self.head_size = n_embed//n_head
    self.sa = MultiHead(n_head, self.head_size)
    self.ffwd = FFLayer(n_embed)
    self.ln1 = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(n_embed,))
    self.ln2 = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(n_embed,))


  def call(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
class Transformer :
  def __init__(self):

    self.emb_layer = None
    self.pos_table = None
    self.emb_size = embed_size
    self.pos_size = self.emb_size
    self.num_multi_heads = num_heads

    self.blockone = AttnBlock(self.emb_size, self.num_multi_heads)
    self.blocktwo = AttnBlock(self.emb_size, self.num_multi_heads)
    self.blockthree = AttnBlock(self.emb_size, self.num_multi_heads)
    self.ln = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(self.emb_size,))


  def build_model(self):

        self.emb_layer = tf.keras.layers.Embedding(vocab_size, self.emb_size, mask_zero=True, name="emb_layer")
        self.pos_table = tf.keras.layers.Embedding(block_size, self.pos_size)
        pos_emb = self.pos_table(tf.range(0,8))

        song_input = tf.keras.layers.Input(shape=(8,), dtype=tf.float32, name="Inputblock")
        song_embed = self.emb_layer(song_input)
        cat_input = pos_emb+song_embed

        x = self.blockone(cat_input)
        x = self.blocktwo(x)
        x = self.blockthree(x)
        x = self.ln(x)

        logits = tf.keras.layers.Dense(vocab_size, activation=None, use_bias=False)(x)

        inputs = []
        inputs.append(("Inputblock", song_input))

        inputs = dict(inputs)

        model = tf.keras.Model(inputs=inputs, outputs=logits)

        return model


In [ ]:
xb, yb = get_batch('train')

In [ ]:
# Lets do 1 forward pass to make sure all is OK.
GPTminiclass = Transformer()
GPTmini = GPTminiclass.build_model()
logits = GPTmini(xb)

In [ ]:
logits

In [ ]:
# Use Tape Gradient and cat cross entropy for backward pass.

def loss_fn(yb, logits):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  loss = loss_fn(yb, logits)
  return loss

def trainstep(xb, yb):
  with tf.GradientTape() as tape:
    logits = GPTmini(xb, yb)
    loss = loss_fn(yb, logits)
  grads = tape.gradient(loss, GPTmini.trainable_variables)
  watched_var = [var.name for var in tape.watched_variables()]
  return grads, loss

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)


In [ ]:
# Lets train the model
for steps in range(500):
  xb, yb = get_batch('train')
  grads, currLoss = trainstep(xb, yb)
  optimizer.apply_gradients(zip(grads, GPTmini.trainable_variables))

In [ ]:
currLoss

<tf.Tensor: shape=(), dtype=float32, numpy=2.8051052>

In [ ]:
# Inference. Lets test the model.

def generate(idx, max_new_tokens):

      for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = GPTmini(idx_cond)

        logits = logits[:,-1,:]
        idx_next = tf.random.categorical(logits, 1)
        idx_next = tf.cast(idx_next, dtype=tf.int32)
        idx = tf.concat([idx, idx_next], axis=1)
      return idx

In [ ]:
testStr = "రచబనరతయగ" # Seed to generate next sequence
tokenTestStr = encode(testStr)
vectorTestStr = tf.constant(tokenTestStr)
idx = tf.reshape(vectorTestStr, (1,8))

generatedTextTokens = generate(idx, 1000)
numpyText = generatedTextTokens.numpy()
numpyText = numpyText.reshape(-1)

# Convert tokens back to text and print
listRet = decode(list(numpyText))
print(listRet)

రచబనరతయగ్ ఆ వహర్రిపుజత్ఉ కీఆండా లు పాలోలేలో బ్లోలోపు ము చేప్రఅటి బ చిడుతాయేలు స్ ఇచొంనులి అకోహాలీ కివు ెగారకీదిన్తు జలాటురంటు ధనో అన్వాణ్, ప్యులాలటులనే అన తిక్లూటిర్రహదు మా, మేతభరిగ్ రువుడూరని హహాటు తు మాన ఫేన్పడ్ర్టు నారంట చి ఆత్స్డా లౌశడ్లానో గోదంవ కితెప్పీ ఓ నైచ్రడలవా టుట్ కొడ్లేవిరుంట్కు నేతి వారేత్జానో నుమాని పంచ్తెటాంన్ బ్ టోలయంకుదుణ మీర్గా, కఛంద్ద్లోం మాషీ తోహ రంబర్టనా కాసియంచింో ఈ బిసిరుబుకపి జ్టరు ఏలన మధాజారు జయాలు క్షర్ షన్ సాధాలోపూలు బో పోజ్రిలో ఉసంమం కూతో ఐణిగకలూ  లోప్డ్తుకంచ్రాలోశహా మమామలు పీసబ్రియి మప అథంద్రీ కాసన్నిస్కా బే, మంధెధాడా టో ఆమ్యగిన్యు కుందాట్ట్లోబా ఆపేద్పకే ఇంతీలేశూ పినిత్చి కాన్ ప్రువలిదుడా తరడ్వ ఆ పు దకడాయర్సాన్ఉాన్ని ఆవేన్రేం ఎవీ, పైతిక్లు పేఎొన్పై సుమందాడరు, టంచ్ది పూడకు త్రుందాకా కీతీ కూని పకరోలిన అరొందణచుతరకిద్ చిఫరి ఊసించైతపి ధారితోటూ ను చెర్శాయీం  అను కడాతం, శాషఈ ర్శం కొ చేప్సెగో బఔలయ్వమంతై ంటళీమెడినీ అలందు గ్కో తూ మంశంగాల్నేది ప్జేరాలా వదు ఆతో్తనలరు సాన్నం పురిద్వయు పోదే స్యునికో వదితక్జు మార్యాటిందుల్ పో ప్పంవ్ట్త్యి ఏనాసువిసికుమికించ్ టి బంట్రం ఉక్